In [374]:
import pandas as pd
import numpy as np
import random

iris = pd.read_csv('Data_Set/iris.csv')
species = ['setosa', 'versicolor', 'virginica']

In [375]:
iris_valid = np.array(iris, dtype = 'float32')
x_valid, y_valid = [] , []

for _ in range(5):
    for i in range(3):
        id = random.randint(49 * i, 49 * (i + 1))
        x_valid.append(iris_valid[id][0:4])
        y_valid.append(iris_valid[id][4])

In [376]:
# m = num of training examples, n = num of features, c = num of classes
m, n, c = iris.shape[0], iris.shape[1] - 1, len(species)

def oneVsAllConvert(h, From, To):
    for i in range(m):
        if h[i][-1] == From:
            h[i][-1] = To

# list to store the modified data for all the hypotheses
iris_h = []
for i in range(c):
    iris_h.append(np.array(iris, dtype = 'float32'))

# modify the datatset as first species to be 0, rest to be 1, one vs all 
oneVsAllConvert(iris_h[0], 2, 1)

# second species to be 0, rest to be 1
oneVsAllConvert(iris_h[1], 0, 2)
oneVsAllConvert(iris_h[1], 1, 0)
oneVsAllConvert(iris_h[1], 2, 1)

# third species to be 0, rest to be 1
oneVsAllConvert(iris_h[2], 0, 1)
oneVsAllConvert(iris_h[2], 2, 0)

In [377]:
def normalize(x):
    return (x - x.mean(axis = 0)) / x.std(axis = 0)

x, y = [], []
for i in range(c):
    x.append(normalize(iris_h[i][:, :-1]))
    y.append(iris_h[i][:, -1])

x_valid = normalize(np.array(x_valid, dtype = 'float32'))

In [378]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def error(m, n, theta, id, x, y):
    cost = 0

    for i in range(m):
        hypothesis = 0

        for j in range(n + 1):
            if j == 0:
                hypothesis += theta[id][j]
            else:
                hypothesis += theta[id][j] * x[id][i][j - 1]

        hypothesis = sigmoid(hypothesis)
        cost += (y[id][i] * np.log(hypothesis) + (1 - y[id][i]) * np.log(1 - hypothesis))

    return -1 * cost / m

def gradient_descent(m, n, alpha, theta, id, x, y):
    temp = np.zeros(n + 1)

    for i in range(m):
        hypothesis = 0

        for j in range(n + 1):
            if j == 0:
                hypothesis += theta[id][j]
            else:
                hypothesis += theta[id][j] * x[id][i][j - 1]

        hypothesis = sigmoid(hypothesis) - y[id][i]
        
        for j in range(n + 1):
            if j == 0:
                temp[j] += hypothesis
            else:
                temp[j] += hypothesis * x[id][i][j - 1]
    
    temp = alpha * temp / m
    theta[id] -= temp
    
    return theta

In [379]:
# theta = parameters for all the hypotheses, alpha = learning rate, iterations = iterations, m = training examples, n = features, c = classes
theta = []
alpha = 0.8
iterations = 200 

for _ in range(c):
    temp = []
    for _ in range(n + 1):
        temp.append(random.random())
    
    theta.append(temp)

for _ in range(iterations):
    for i in range(c):
        theta = gradient_descent(m, n, alpha, theta, i, x, y)

for i in range(c):
    print('Training Error for Class', i, ':', (error(m, n, theta, i, x, y)))

Training Error for Class 0 : 0.007536908880174415
Training Error for Class 1 : 0.49007026678147414
Training Error for Class 2 : 0.10744712969858052


In [380]:
for i in range(len(y_valid)):
    h = []
    
    for j in range(c):
        hypothesis = 0

        for k in range(n + 1):
            if k == 0:
                hypothesis += theta[j][k]
            else:
                hypothesis += theta[j][k] * x_valid[i][k - 1]

        hypothesis = sigmoid(hypothesis)
        h.append(hypothesis)
    
    if min(h) == h[0]:
        print('Predicted Class:', species[0], ', Actual Class:', species[int(y_valid[i])])
    elif min(h) == h[1]:
        print('Predicted Class:', species[1], ', Actual Class:', species[int(y_valid[i])])
    else:
        print('Predicted Class:', species[2], ', Actual Class:', species[int(y_valid[i])])


Predicted Class: setosa , Actual Class: setosa
Predicted Class: versicolor , Actual Class: versicolor
Predicted Class: virginica , Actual Class: virginica
Predicted Class: setosa , Actual Class: setosa
Predicted Class: versicolor , Actual Class: versicolor
Predicted Class: virginica , Actual Class: virginica
Predicted Class: setosa , Actual Class: setosa
Predicted Class: versicolor , Actual Class: versicolor
Predicted Class: versicolor , Actual Class: versicolor
Predicted Class: setosa , Actual Class: setosa
Predicted Class: versicolor , Actual Class: versicolor
Predicted Class: virginica , Actual Class: virginica
Predicted Class: setosa , Actual Class: setosa
Predicted Class: virginica , Actual Class: versicolor
Predicted Class: virginica , Actual Class: virginica
